# Convert the RAW dataset to TFRecord

In [1]:
import os
import tensorflow as tf
from PIL import Image
from pascal_voc_writer import Writer

In [2]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('database', 'GTSDB',
                           'Training data directory')
tf.app.flags.DEFINE_string('train_dir', './train',
                           'Training data directory')
tf.app.flags.DEFINE_string('val_dir', './val',
                           'Validation data directory')
tf.app.flags.DEFINE_string('test_dir', './test',
                           'Validation data directory')
tf.app.flags.DEFINE_string('csv_file', './gt.txt',
                           'Path to the csv annotation file')
tf.app.flags.DEFINE_string('xml_dir', './converted/annotations',
                           'Validation data directory')
tf.app.flags.DEFINE_string('converted_img_dir', './converted/images',
                           'Converted data directory')

In [ ]:
def get_ppm_files(data_dir):
    """"Reads addresses and labels from `data_dir`.
    The `tf.gfile.Glob` module return a list of paths matching a pathname pattern
    
    Args:
        data_dir = Path to the `input image folder`.
    
    Returns:
        sorted_matching_files: list, address of each picture in the train folder.
    """
    cur_dir = os.getcwd()
    new_dir = os.chdir(data_dir)
    
    ppm_file = '*.ppm'
    matching_files = tf.gfile.Glob(ppm_file)
    sorted_matching_files = sorted(matching_files)
    
    # Go back to the original directory
    os.chdir(cur_dir)
    
    return sorted_matching_files


def convert_ppm_to_jpg(input_dir, filename, converted_img_dir):
    converted_filename = filename.replace('.ppm', '.jpg')
    if not os.path.isdir(converted_dir):
        os.makedirs(converted_dir)
    
    image = Image.open(os.path.join(input_dir, filename))
    converted_img_path = os.path.join(converted_dir, converted_filename)
    image.save(converted_img_path, mode='JPEG', quality=100, optimize=True, progressive=True)
    
    return None
    
    
def main(unused_argv):
    filenames = get_ppm_files(FLAGS.train_dir)
    for filename in filenames:
        convert_ppm_to_jpg(FLAGS.train_dir, filename, FLAGS.converted_img_dir)


if __name__ == '__main__':
    tf.app.run()

## Convert *.csv to *.xml(s)

The default csv annotation format is: `<filename>;<xmin;ymin;xmax;ymax>;<class_id>`

For example, this is how we will process csv file(s) and convert it into xml(s)

```python
obj_attributes_string = '00000.ppm;774;411;815;446;11'

obj_attributes_split = input_str.split(';')

writer = Writer(obj_attributes_split[0], 300, 300, database='Unknown')
writer.addObject(
    obj_attributes_split[-1],
    obj_attributes_split[1],
    obj_attributes_split[2],
    obj_attributes_split[3],
    obj_attributes_split[4])
writer.save('out.xml')
```

In [7]:
def create_xml_annotation(database, csv_file_path, image_dir, xml_dir):
    if not os.path.isdir(xml_dir):
            os.makedirs(xml_dir)

    annotation_content = []
    with open(csv_file_path, 'r') as f:
        for line in f:
            annotation_content.append(line)

    rows = len(annotation_content)
    curr_row = 0
    while curr_row < rows:
        obj_attributes_string = annotation_content[curr_row]
        obj_attributes_split = obj_attributes_string.split(';')

        image_path = os.path.join(image_dir, obj_attributes_split[0])
        image = Image.open(image_path)
        width, height = image.size

        writer = Writer(obj_attributes_split[0], width, height, database)
        # Need to plus 1 to the class id
        # because the csv annotations start from 0,
        # while label map ids should be >= 1
        writer.addObject(int(obj_attributes_split[-1])+1,
                         obj_attributes_split[1],
                         obj_attributes_split[2],
                         obj_attributes_split[3],
                         obj_attributes_split[4])

        sub_obj = []
        for j in range(curr_row+1, rows):
            if annotation_content[j].startswith(obj_attributes_split[0]):
                sub_obj.append(annotation_content[j])
            else:
                break

        print(f'{obj_attributes_split[0]}-->[{len(sub_obj)+1}]')
        if len(sub_obj) > 0:
            curr_row += len(sub_obj) + 1
            for s_line in sub_obj:
                s_a = s_line.split(';')
                writer.addObject(int(s_a[-1])+1, s_a[1], s_a[2], s_a[3], s_a[4])
        else:
            curr_row += 1
        xml_name = obj_attributes_split[0].replace('.jpg', '.xml')
        writer.save(os.path.join(xml_dir, xml_name))

    
def main(unused_argv):
    create_xml_annotation(FLAGS.database, FLAGS.csv_file, FLAGS.converted_dir, FLAGS.xml_dir)

    
if __name__ == '__main__':
    tf.app.run()

00000.jpg-->[1]
00001.jpg-->[3]
00002.jpg-->[1]
00003.jpg-->[3]
00004.jpg-->[2]
00005.jpg-->[1]
00006.jpg-->[1]
00007.jpg-->[1]
00008.jpg-->[2]
00009.jpg-->[1]
00010.jpg-->[1]
00011.jpg-->[2]
00012.jpg-->[1]
00013.jpg-->[1]
00014.jpg-->[1]
00015.jpg-->[1]
00016.jpg-->[1]
00017.jpg-->[2]
00018.jpg-->[1]
00019.jpg-->[2]
00020.jpg-->[1]
00021.jpg-->[1]
00022.jpg-->[1]
00023.jpg-->[3]
00024.jpg-->[1]
00025.jpg-->[2]
00026.jpg-->[1]
00027.jpg-->[2]
00028.jpg-->[2]
00029.jpg-->[1]
00030.jpg-->[1]
00031.jpg-->[2]
00032.jpg-->[2]
00033.jpg-->[1]
00034.jpg-->[1]
00035.jpg-->[2]
00036.jpg-->[2]
00037.jpg-->[1]
00038.jpg-->[1]
00039.jpg-->[2]
00040.jpg-->[2]
00041.jpg-->[1]
00042.jpg-->[1]
00043.jpg-->[3]
00044.jpg-->[1]
00045.jpg-->[1]
00046.jpg-->[1]
00047.jpg-->[1]
00048.jpg-->[3]
00049.jpg-->[2]
00050.jpg-->[2]
00051.jpg-->[1]
00052.jpg-->[2]
00053.jpg-->[2]
00054.jpg-->[4]
00055.jpg-->[1]
00056.jpg-->[1]
00057.jpg-->[1]
00058.jpg-->[1]
00059.jpg-->[1]
00060.jpg-->[2]
00061.jpg-->[1]
00062.jp

00638.jpg-->[1]
00639.jpg-->[1]
00640.jpg-->[2]
00641.jpg-->[1]
00642.jpg-->[2]
00644.jpg-->[1]
00645.jpg-->[1]
00646.jpg-->[2]
00647.jpg-->[1]
00648.jpg-->[1]
00649.jpg-->[1]
00650.jpg-->[2]
00651.jpg-->[2]
00653.jpg-->[2]
00654.jpg-->[1]
00655.jpg-->[2]
00656.jpg-->[1]
00657.jpg-->[1]
00658.jpg-->[1]
00659.jpg-->[1]
00660.jpg-->[1]
00661.jpg-->[1]
00662.jpg-->[1]
00663.jpg-->[1]
00664.jpg-->[2]
00665.jpg-->[2]
00667.jpg-->[1]
00668.jpg-->[2]
00669.jpg-->[1]
00670.jpg-->[1]
00671.jpg-->[2]
00672.jpg-->[2]
00673.jpg-->[1]
00674.jpg-->[1]
00675.jpg-->[3]
00678.jpg-->[1]
00679.jpg-->[2]
00680.jpg-->[2]
00681.jpg-->[1]
00682.jpg-->[3]
00683.jpg-->[2]
00685.jpg-->[1]
00686.jpg-->[1]
00688.jpg-->[2]
00689.jpg-->[2]
00690.jpg-->[1]
00691.jpg-->[2]
00692.jpg-->[1]
00693.jpg-->[1]
00694.jpg-->[1]
00695.jpg-->[1]
00696.jpg-->[1]
00697.jpg-->[1]
00698.jpg-->[1]
00699.jpg-->[1]
00700.jpg-->[1]
00701.jpg-->[2]
00702.jpg-->[1]
00703.jpg-->[2]
00704.jpg-->[1]
00705.jpg-->[1]
00706.jpg-->[1]
00707.jp

SystemExit: 